In [82]:
import os
from pymongo import MongoClient
import json
from dotenv import load_dotenv
# Langchain load data
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
### LLMs - Langchain
import openai
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
### Summary chain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain, MapReduceChain, load_summarize_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.document_loaders import TextLoader

In [83]:
def get_directories(path: str):
    """
    Get all directory names under the specified path.
    
    :param path: The root directory path to search.
    :return: A list of directory names.
    """
    try:
        return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    except FileNotFoundError:
        print(f"Error: The path '{path}' does not exist.")
        return []
    except PermissionError:
        print(f"Error: Permission denied for path '{path}'.")
        return []
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
def get_ids_with_condition(collection,condition):
    """
    Retrieve an array of `_id` values where "sent-out" is "no data".

    :param collection: MongoDB collection object.
    :return: List of `_id` values matching the condition.
    """
#     condition = {"sent-out": "no data"}
    result = collection.find(condition, {"_id": 1})  # Retrieve only the `_id` field
    return [doc["_id"] for doc in result]

def create_sensitive_data_prompt(traffic_data: str):
    """
    Analyze Android app network traffic to detect sensitive data leaks.
    
    :param traffic_data: Raw traffic data (JSON or text).
    :return: A formatted prompt for an LLM to analyze the data.
    """
    # Escape curly braces to prevent formatting errors
    escaped_traffic_data = traffic_data.replace("{", "{{").replace("}", "}}").replace("\"", "\\\"")

    sensitive_data_prompt = PromptTemplate(
        input_variables=["traffic_data"],
        template="""
        You are a cybersecurity expert specializing in **privacy and data security**. 
        Your task is to analyze the outgoing network traffic of an Android application 
        and identify any **sensitive information** that could pose privacy risks.

        ### **Given Traffic Data:**
        ```
        {traffic_data}
        ```

        ### **Your Task:**
        1. Identify any **sensitive data** being transmitted, including but not limited to:
           - **Personal Identifiable Information (PII)**: Email, phone number, IP address, user credentials.
           - **Device Information**: Model, OS version, manufacturer, fingerprint, MCC/MNC.
           - **Authentication Data**: Firebase authentication tokens, API keys, session IDs.
           - **Tracking & Advertising Data**: Firebase installation ID, advertising ID, package name.
           - **Location & Network Data**: Country, locale, timezone, network connection type.
           - **Any other potentially sensitive data** that could lead to privacy risks.

        2. Evaluate the potential **privacy risks** associated with each piece of sensitive data.

        ---

        ### **Expected JSON Output Format:**
        ```json
        {{
            "sensitive_data_detected": [
                {{
                    "type": "Device ID",
                    "value": "...",
                    "risk": "This can be used to track users across applications."
                }},
                {{
                    "type": "Firebase Authentication Token",
                    "value": "...",
                    "risk": "May allow unauthorized access if leaked."
                }}
            ],
        }}
        ```
        
        If you **cannot detect any sensitive information**, the expected output is:
        ```json
        {{
            "sensitive_data_detected": "no sent-out sensitive data"
        }}
        ```
        
        Please analyze the given traffic data and return a **structured JSON output** as described.
        """
    )
    
    formatted_prompt = sensitive_data_prompt.format(traffic_data=escaped_traffic_data)
    return formatted_prompt

def load_traffic_data(directory_path: str):
    """
    Load traffic logs from the specified directory.
    
    :param directory_path: Path to the directory containing traffic log files.
    :return: A list of loaded traffic data as strings.
    """
    loader = DirectoryLoader(directory_path, glob="**/*.txt", loader_cls=TextLoader)
    docs = loader.load()
    
    traffic_logs = []
    for doc in docs:
        content = doc.page_content.strip()
        # Check if the content is JSON
        try:
            traffic_json = json.loads(content)
            traffic_logs.append(json.dumps(traffic_json, indent=2))
        except json.JSONDecodeError:
            # If not JSON, store as raw text
            traffic_logs.append(content)
    
    return traffic_logs
def write_log(file_path, content):
    """Append content to a log file."""
    with open(file_path, "a", encoding="utf-8") as log_file:
        log_file.write(content + "\n")
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
# def extract_output_text_as_dict(response):
#     """
#     Extracts the content from an AIMessage object, cleans it, and converts it into a dictionary.

#     :param response: The AIMessage object containing the response.
#     :return: A dictionary parsed from the response, or None if parsing fails.
#     """
#     # Ensure response is an AIMessage and extract content
#     output_text = getattr(response, "content", "")

#     # Remove code block markers (```json and ```) if present
#     if output_text.startswith("```json"):
#         output_text = output_text[len("```json"):].strip("`").strip()
    
#     try:
#         # Convert the cleaned JSON string into a dictionary
#         output_dict = json.loads(output_text)
#         return output_dict
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON: {e}")
#         return None

# def append_sensitive_data(data_dict, sent_out_arr):
#     """
#     Append each dictionary under 'sensitive_data_detected' into sent_out_arr.
#     If 'sensitive_data_detected' is a string, append it as a dictionary.

#     :param data_dict: Dictionary containing sensitive data under "sensitive_data_detected".
#     :param sent_out_arr: List to append the extracted data.
#     """
#     if "sensitive_data_detected" in data_dict:
#         detected_data = data_dict["sensitive_data_detected"]

#         if isinstance(detected_data, list):  
#             # If it's a list, extend the list with individual dictionaries
#             sent_out_arr.extend(detected_data)
#         elif isinstance(detected_data, str):  
#             # If it's a string, append it as a dictionary
#             sent_out_arr.append({"sensitive_data_detected": detected_data})

def extract_output_text_as_dict(response):
    """
    Extracts the 'output_text' from the response, cleans it, and converts it into a dictionary.

    :param response: The response dictionary containing 'output_text'.
    :return: A dictionary parsed from the 'output_text', or an empty dict if parsing fails.
    """
    if not response or not hasattr(response, "content"):  # Ensure response is valid
        print("Error: Invalid response object.")
        return {}

    output_text = response.content  # Assuming response.content holds the JSON string

    # Remove code block markers (```json and ```) if present
    if output_text.startswith("```json"):
        output_text = output_text[len("```json"):].strip("`").strip()

    try:
        # Convert the cleaned JSON string into a dictionary
        output_dict = json.loads(output_text)
        return output_dict
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return {}  # Return an empty dictionary instead of None


def append_sensitive_data(data_dict, sent_out_arr):
    """
    Append each dictionary under 'sensitive_data_detected' into sent_out_arr.
    If 'sensitive_data_detected' is a string, append it as a dictionary.

    :param data_dict: Dictionary containing sensitive data under "sensitive_data_detected".
    :param sent_out_arr: List to append the extracted data.
    """
    if not isinstance(data_dict, dict):  # Ensure data_dict is valid
        print("Error: data_dict is None or not a dictionary.")
        return  # Exit function safely

    if "sensitive_data_detected" in data_dict:
        detected_data = data_dict["sensitive_data_detected"]

        if isinstance(detected_data, list):  
            # If it's a list, extend the list with individual dictionaries
            sent_out_arr.extend(detected_data)
        elif isinstance(detected_data, str):  
            # If it's a string, append it as a dictionary
            sent_out_arr.append({"sensitive_data_detected": detected_data})


def remove_duplicates(data_list):
    """
    Removes dictionaries with duplicate 'type' and 'value' fields while keeping the first occurrence.
    If a dictionary does not contain 'type' and 'value', it is kept unchanged.

    :param data_list: List of dictionaries containing 'type' and 'value' keys.
    :return: A list with unique dictionaries based on 'type' and 'value'.
    """
    seen = set()
    unique_list = []

    for item in data_list:
        # Check if the dictionary contains "type" and "value"
        if "type" in item and "value" in item:
            # Convert 'value' to a string to ensure it's hashable (for sets)
            value_str = str(item["value"])
            identifier = (item["type"], value_str)

            # If it's unique, add to the set and list
            if identifier not in seen:
                seen.add(identifier)
                unique_list.append(item)
        else:
            # Keep items that don't have "type" and "value" unchanged
            unique_list.append(item)

    return unique_list
def remove_duplicates_by_value(data_list):
    """
    Removes dictionaries with duplicate 'value' fields while keeping the first occurrence.
    If a dictionary does not contain 'value', it is kept unchanged.

    :param data_list: List of dictionaries containing 'value' keys.
    :return: A list with unique dictionaries based on 'value'.
    """
    seen_values = set()
    unique_list = []

    for item in data_list:
        # Check if "value" key exists in the dictionary
        if "value" in item:
            # Convert value to a string to handle non-hashable values like dicts
            value_str = str(item["value"])
            
            if value_str not in seen_values:
                seen_values.add(value_str)
                unique_list.append(item)
        else:
            # Keep items that don't have "value" unchanged
            unique_list.append(item)

    return unique_list


def update_mongodb_element(collection, _id, update_keyword, update_value):
    """
    Updates a MongoDB document based on the given _id value.
    
    :param collection: The MongoDB collection.
    :param _id: The unique identifier of the document to update.
    :param update_keyword: The field to update in the document.
    :param update_value: Either a string or a list of dictionaries.
    :return: The result of the update operation.
    """
    # Validate that update_value is either a string or a list of dictionaries
    if not isinstance(update_value, (str, list)):
        raise ValueError("update_value must be either a string or a list of dictionaries")

    # If it's a list, ensure it contains only dictionaries
    if isinstance(update_value, list) and not all(isinstance(item, dict) for item in update_value):
        raise ValueError("update_value as a list must contain only dictionaries")

    # MongoDB Update Query
    update_query = {"$set": {update_keyword: update_value}}

    # Perform update
    result = collection.update_one({"_id": _id}, update_query)

    # Return update result
    return result

In [84]:
root_traffic_directory_path = r"E:\wearable-capture-traffic\wearable-app"
# MONGO DB
mongoDB_uri = 'mongodb://192.168.1.14:27017'
mongoDB_database = 'wearable-project' 
mongoDB_collection = 'wearable-app'

In [85]:
array_traffic_directory = get_directories(root_traffic_directory_path)
print(array_traffic_directory)

['adarshurs.android.vlcmobileremote', 'app.EnForma.fitcoapp.net', 'app.entrenamientogenetico.android.fitcoapp.net', 'app.f4square.fitness', 'app.fit.meet', 'app.growwithjo.diet.fitness', 'app.krowdfit', 'app.LomasGym.fitcoapp.net', 'app.mfit.personal', 'app.midfit.train.own', 'app.mlfitness.diet.workout', 'app.MOVE.fitcoapp.net', 'app.weightfit.android', 'appnativo.navysportcenter.fitcoapp.net', 'at.radioplayer.android', 'biz.mobidev.mdplayer', 'blacknote.amazfitmaster', 'br.com.fitastic.appaluno', 'br.com.mufitness.station.aluno', 'br.com.personalfitapp', 'br.com.tecnofit.mobileapp', 'br.com.w12.bluefit', 'br.com.w12.heyfit', 'br.com.w12.redfit', 'br.com.w12.skyfitapp', 'calorie.pedo.fitbit', 'club.nfit', 'club.skllzz.fitness', 'clubwise.com.fitsense', 'cn.fitdays.fitdays', 'co.macrofit.macroFit', 'coach.zing.fitness', 'com.abhijitvalluri.android.fitnotifications', 'com.abyx.fit.connect', 'com.acmeaom.android.myradar', 'com.acquasys.contrack', 'com.actofitSmartScale', 'com.adi.remote.

In [86]:
# Connect to the MongoDB collection
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)
condition = {"sent-out": "no data"}
no_traffic_package_name = get_ids_with_condition(collection,condition)
print(no_traffic_package_name)
print(len(no_traffic_package_name))

['com.reflive.reflive', 'com.atlaswearables.ionic', 'com.acquasys.contrack', 'com.daniel.apps.wpe', 'net.ddns.axiiim.wearphonelock', 'org.uguess.android.sysinfo.wear', 'com.ericfabreu.wearflashcards', 'com.lilc103.buttonbattle', 'com.overseasolutions.waterapp.app', 'tk.deuchnord.phonejuicechecker', 'de.softxperience.android.quickprofiles', 'com.piggeh.flip', 'kr.co.openit.openrider', 'com.gin.shoppinglist', 'com.ccswe.SmokingLog', 'net.sarmady.akhbarak', 'org.sombrenuit.dk.redfeed.free', 'com.shinayser.xwingmanager', 'com.aheqab.salahwear', 'eu.power_switch', 'com.yannik.wear.anki', 'com.muzzley', 'net.homeip.thkerkmann.einkaufsliste', 'com.pimp.calculator', 'at.radioplayer.android', 'com.dimlix.phrase', 'com.cm.wearintervaltimer', 'com.droibit.findwear', 'com.pasha.music977', 'com.directed.android.smartstart', 'com.clearskyapps.fitnessfamily.Run5K', 'com.google.android.deskclock', 'com.shazam.android', 'de.komoot.android', 'com.gp.android.fitproject.colorwalk', 'com.planetfitness', 'c

In [87]:
array_traffic_package_name = list(set(array_traffic_directory) - set(no_traffic_package_name))
print(array_traffic_package_name)
print(len(array_traffic_package_name))

['steptracker.healthandfitness.walkingtracker.pedometer', 'com.rokontrol.android', 'com.fitplanapp.fitplan', 'com.ktmusic.geniemusic', 'com.mimm.mifitness', 'org.moffitt.brandedpatientportal.play', 'com.crrepa.band.havefit', 'com.trainerize.witfit', 'com.lifetimefitness.interests.fitness', 'com.naukriGulf.app', 'com.india.fitpass', 'com.fitnow.app', 'com.shapehq.fitnessworld', 'com.estela', 'com.cisco.im', 'fitapp.fittofit', 'com.actofitSmartScale', 'com.kiss.keeper', 'com.fitonomy.muscle.man', 'com.airofit_app', 'fitness.app.women.female.workout', 'digifit.android.virtuagym.pro.ymcaofgreatertoronto', 'com.zelpasfitnesswraps.getabiggerbootyin7minutesbuttocksworkoutathome', 'ru.disav.befit', 'com.proyecto.fitup.tgcustom', 'pl.tweeba.mobile.learning.english', 'scope.fitnessfirstmena', 'com.proyecto.beonefitnesssport.tgcustom', 'com.nordlane.app_fitclubfinland_fi', 'com.simplyfitme.weightloss', 'com.vgfit.shefit', 'com.mictale.gpsessentials', 'com.proyecto.fit.tgcustom', 'com.oplayer.good

In [88]:
#. Chat model
# Setup model
# Load environment variables
load_dotenv()

# Retrieve API key
api_key = os.getenv("OPENAI_API_KEY")
# Ensure the API key is correctly set
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=api_key  # Ensure you explicitly pass the API key
)

In [89]:
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)

In [90]:
for i in range(0,len(array_traffic_package_name)):
# for i in range(0,2):
    print("------------------- Loop-"+str(i)+"-------------------")
    check_package_name = array_traffic_package_name[i]
    print("check_package_name: ",check_package_name)
    check_directory_path = root_traffic_directory_path+"\\"+check_package_name+"\\final-traffic"
    print("check_directory_path: ",check_directory_path)
    
    log_file_path = f"log-{check_package_name}.log"  # Create log filename
    # Write initial log info
    write_log(log_file_path, f"------------------- Loop-{i} -------------------")
    write_log(log_file_path, f"check_package_name: {check_package_name}")
    traffic_data_arr = load_traffic_data(check_directory_path)
    write_log(log_file_path, f"check_directory_path: {check_directory_path}")
    sent_out_arr = []
    for j in range(len(traffic_data_arr)):
        traffic_data = traffic_data_arr[j]
        write_log(log_file_path, f"===================== Traffic-{j} =====================")
        formatted_prompt = create_sensitive_data_prompt(traffic_data)
        write_log(log_file_path, f"prompt: {formatted_prompt}")
        output = llm.invoke(formatted_prompt)
        write_log(log_file_path, f"******** LLM response ********\n{output.content}")
        output_dict = extract_output_text_as_dict(output)
        write_log(log_file_path, f"Extracted output as dict: {output_dict}")
        append_sensitive_data(output_dict,sent_out_arr)
#     print(sent_out_arr)
    write_log(log_file_path, f"===================== Origin Sent Out Array =====================")
    print("Length sent_out_arr: ", len(sent_out_arr))
    write_log(log_file_path, f"Sent out array: {sent_out_arr}")
    remove_duplicate_sent_out_arr = remove_duplicates(sent_out_arr)
    final_sent_out_arr = remove_duplicates_by_value(remove_duplicate_sent_out_arr)
    write_log(log_file_path, f"===================== Final Sent Out Array =====================")
    print("Length final sent_out_arr: ", len(final_sent_out_arr))
    write_log(log_file_path, f"Final Sent Out Array: {final_sent_out_arr}")
    update_result_status = update_mongodb_element(collection, check_package_name, "sent-out", final_sent_out_arr)
    if update_result_status.modified_count > 0:
        print(f"Successfully updated document with list for _id: {check_package_name}")
    else:
        print(f"No document found or updated for _id: {check_package_name} (list update)")
#     break

------------------- Loop-0-------------------
check_package_name:  steptracker.healthandfitness.walkingtracker.pedometer
check_directory_path:  E:\wearable-capture-traffic\wearable-app\steptracker.healthandfitness.walkingtracker.pedometer\final-traffic
Length sent_out_arr:  26
Length final sent_out_arr:  25
Successfully updated document with list for _id: steptracker.healthandfitness.walkingtracker.pedometer
------------------- Loop-1-------------------
check_package_name:  com.rokontrol.android
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.rokontrol.android\final-traffic
Length sent_out_arr:  3
Length final sent_out_arr:  3
Successfully updated document with list for _id: com.rokontrol.android
------------------- Loop-2-------------------
check_package_name:  com.fitplanapp.fitplan
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.fitplanapp.fitplan\final-traffic
Length sent_out_arr:  31
Length final sent_out_arr:  29
Successfully updated docume

Length sent_out_arr:  6
Length final sent_out_arr:  6
Successfully updated document with list for _id: pl.tweeba.mobile.learning.english
------------------- Loop-26-------------------
check_package_name:  scope.fitnessfirstmena
check_directory_path:  E:\wearable-capture-traffic\wearable-app\scope.fitnessfirstmena\final-traffic
Length sent_out_arr:  5
Length final sent_out_arr:  5
Successfully updated document with list for _id: scope.fitnessfirstmena
------------------- Loop-27-------------------
check_package_name:  com.proyecto.beonefitnesssport.tgcustom
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.proyecto.beonefitnesssport.tgcustom\final-traffic
Length sent_out_arr:  19
Length final sent_out_arr:  17
Successfully updated document with list for _id: com.proyecto.beonefitnesssport.tgcustom
------------------- Loop-28-------------------
check_package_name:  com.nordlane.app_fitclubfinland_fi
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.nor

Length sent_out_arr:  23
Length final sent_out_arr:  23
Successfully updated document with list for _id: com.netpulse.mobile.morefitness
------------------- Loop-50-------------------
check_package_name:  com.biostatusfit.www
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.biostatusfit.www\final-traffic
Length sent_out_arr:  27
Length final sent_out_arr:  25
Successfully updated document with list for _id: com.biostatusfit.www
------------------- Loop-51-------------------
check_package_name:  com.Female.Fitness.cardio.wellness
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.Female.Fitness.cardio.wellness\final-traffic
Error decoding JSON: Extra data: line 4 column 1 (char 64)
Length sent_out_arr:  0
Length final sent_out_arr:  0
Successfully updated document with list for _id: com.Female.Fitness.cardio.wellness
------------------- Loop-52-------------------
check_package_name:  com.user.livefitt
check_directory_path:  E:\wearable-capture-traffic

Length sent_out_arr:  60
Length final sent_out_arr:  51
Successfully updated document with list for _id: com.gymshark.fitness
------------------- Loop-73-------------------
check_package_name:  digifit.android.virtuagym.pro.levelupgym
check_directory_path:  E:\wearable-capture-traffic\wearable-app\digifit.android.virtuagym.pro.levelupgym\final-traffic
Length sent_out_arr:  10
Length final sent_out_arr:  9
Successfully updated document with list for _id: digifit.android.virtuagym.pro.levelupgym
------------------- Loop-74-------------------
check_package_name:  app.mfit.personal
check_directory_path:  E:\wearable-capture-traffic\wearable-app\app.mfit.personal\final-traffic
Length sent_out_arr:  13
Length final sent_out_arr:  12
Successfully updated document with list for _id: app.mfit.personal
------------------- Loop-75-------------------
check_package_name:  com.joshua.hasfit
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.joshua.hasfit\final-traffic
Length sent_ou

Error decoding JSON: Extra data: line 4 column 1 (char 64)
Length sent_out_arr:  6
Length final sent_out_arr:  6
Successfully updated document with list for _id: net.kosev.dicing
------------------- Loop-98-------------------
check_package_name:  br.com.w12.heyfit
check_directory_path:  E:\wearable-capture-traffic\wearable-app\br.com.w12.heyfit\final-traffic
Length sent_out_arr:  13
Length final sent_out_arr:  8
Successfully updated document with list for _id: br.com.w12.heyfit
------------------- Loop-99-------------------
check_package_name:  com.netpulse.mobile.healthfitnesscenter
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.netpulse.mobile.healthfitnesscenter\final-traffic
Length sent_out_arr:  23
Length final sent_out_arr:  21
Successfully updated document with list for _id: com.netpulse.mobile.healthfitnesscenter
------------------- Loop-100-------------------
check_package_name:  rs.nonstopfitness
check_directory_path:  E:\wearable-capture-traffic\wearable

...

Length sent_out_arr:  4
Length final sent_out_arr:  4
Successfully updated document with list for _id: com.tony.hifitpro
------------------- Loop-418-------------------
check_package_name:  com.huawo.ufit
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.huawo.ufit\final-traffic
Length sent_out_arr:  10
Length final sent_out_arr:  10
Successfully updated document with list for _id: com.huawo.ufit
------------------- Loop-419-------------------
check_package_name:  com.topstep.fitcloudpro
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.topstep.fitcloudpro\final-traffic
Length sent_out_arr:  2
Length final sent_out_arr:  2
Successfully updated document with list for _id: com.topstep.fitcloudpro
------------------- Loop-420-------------------
check_package_name:  com.vfit.vfit
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.vfit.vfit\final-traffic
Length sent_out_arr:  11
Length final sent_out_arr:  11
Successfully updated document

Length sent_out_arr:  7
Length final sent_out_arr:  7
Successfully updated document with list for _id: fit.remotecoach.fitin
------------------- Loop-444-------------------
check_package_name:  com.bhphoto
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.bhphoto\final-traffic
Length sent_out_arr:  17
Length final sent_out_arr:  17
Successfully updated document with list for _id: com.bhphoto
------------------- Loop-445-------------------
check_package_name:  com.bottegasol.com.migym.fitnessconnection
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.bottegasol.com.migym.fitnessconnection\final-traffic
Length sent_out_arr:  47
Length final sent_out_arr:  37
Successfully updated document with list for _id: com.bottegasol.com.migym.fitnessconnection
------------------- Loop-446-------------------
check_package_name:  com.starmax.runmefit
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.starmax.runmefit\final-traffic
Length sent_out_a

Length sent_out_arr:  32
Length final sent_out_arr:  30
Successfully updated document with list for _id: com.fitness22.workout
------------------- Loop-469-------------------
check_package_name:  com.chloeting.fitness
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.chloeting.fitness\final-traffic
Length sent_out_arr:  6
Length final sent_out_arr:  6
Successfully updated document with list for _id: com.chloeting.fitness
------------------- Loop-470-------------------
check_package_name:  com.fitness22.meditation
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.fitness22.meditation\final-traffic
Length sent_out_arr:  6
Length final sent_out_arr:  6
Successfully updated document with list for _id: com.fitness22.meditation
------------------- Loop-471-------------------
check_package_name:  com.basicfit.trainingApp
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.basicfit.trainingApp\final-traffic
Error decoding JSON: Extra data: li

Length sent_out_arr:  5
Length final sent_out_arr:  5
Successfully updated document with list for _id: com.kfaraj.notepad
------------------- Loop-493-------------------
check_package_name:  com.urbandroid.lux
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.urbandroid.lux\final-traffic
Length sent_out_arr:  6
Length final sent_out_arr:  5
Successfully updated document with list for _id: com.urbandroid.lux
------------------- Loop-494-------------------
check_package_name:  com.yesfitclient
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.yesfitclient\final-traffic
Length sent_out_arr:  1
Length final sent_out_arr:  1
Successfully updated document with list for _id: com.yesfitclient
------------------- Loop-495-------------------
check_package_name:  digifit.android.virtuagym.pro.ymcapeninsulademo
check_directory_path:  E:\wearable-capture-traffic\wearable-app\digifit.android.virtuagym.pro.ymcapeninsulademo\final-traffic
Length sent_out_arr:  10
Le

Length sent_out_arr:  3
Length final sent_out_arr:  3
Successfully updated document with list for _id: com.arisdot.gymfit
------------------- Loop-517-------------------
check_package_name:  com.kannergroup.fitnessculture
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.kannergroup.fitnessculture\final-traffic
Length sent_out_arr:  15
Length final sent_out_arr:  14
Successfully updated document with list for _id: com.kannergroup.fitnessculture
------------------- Loop-518-------------------
check_package_name:  com.fitlab.spartan
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.fitlab.spartan\final-traffic
Length sent_out_arr:  14
Length final sent_out_arr:  14
Successfully updated document with list for _id: com.fitlab.spartan
------------------- Loop-519-------------------
check_package_name:  com.bendingspoons.thirtydayfitness
check_directory_path:  E:\wearable-capture-traffic\wearable-app\com.bendingspoons.thirtydayfitness\final-traffic
Length 